In [ ]:
# Download repos with rigt branch name
%cd /content/
!git clone https://github.com/zldrobit/yolov5.git
%cd yolov5
!git checkout tf-android

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 3242, done.
remote: Total 3242 (delta 0), reused 0 (delta 0), pack-reused 3242
Receiving objects: 100% (3242/3242), 8.72 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (2061/2061), done.
/content/yolov5
Branch 'tf-android' set up to track remote branch 'tf-android' from 'origin'.
Switched to a new branch 'tf-android'


In [ ]:
# Download weight files
!./weights/download_weights.sh

100% 14.5M/14.5M [00:00<00:00, 62.6MB/s]

100% 41.9M/41.9M [00:01<00:00, 43.3MB/s]

100% 91.6M/91.6M [00:01<00:00, 60.1MB/s]

100% 170M/170M [00:01<00:00, 100MB/s] 



In [ ]:
# Install requirements
%set_env PYTHONPATH=/content/yolov5
import os 
os.environ['PYTHONPATH'] += ":/content/yolov5" 
import sys 
sys.path.append("/content/yolov5") 

!pip install -r requirements.txt

!pip install tensorflow==2.3.1

env: PYTHONPATH=/content/yolov5
     |████████████████████████████████| 276kB 3.5MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=bb25fa7655de11839f6cdef87edfacc846653541c9415bb9c9612ebdd9199773
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 320.4MB 23kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [ ]:
#Download the COCO dataset for the representative dataset generation
!./data/scripts/get_coco.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   644  100   644    0     0   3220      0 --:--:-- --:--:-- --:--:--  3220
100 68.2M  100 68.2M    0     0  58.2M      0  0:00:01  0:00:01 --:--:-- 74.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0G  100 18.0G    0     0  58.5M      0  0:05:14  0:05:14 --:--:-- 55.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  777M  100  777M    0     0  77.8M      0  0:00:09  0:00:09 --:--:-- 65.4M


By using this tip: https://github.com/ultralytics/yolov5/pull/1127#issuecomment-726702917 

It can run but increadibly slow.

In [ ]:
# Convert models
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 320 --no-tfl-detect --tfl-int8 --source /content/coco/images/train2017 --ncalib 100

2020-11-20 11:17:38.850084: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[320, 320], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/content/coco/images/train2017', tf_nms=False, tfl_int8=True, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-11-20 11:17:41.520347: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-20 11:17:41.530381: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-11-20 11:17:41.530441: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (db48cf74c161): /proc/driver/nvidia/version does not exist
2020-11-20 11:17:41.5

In [ ]:
#Compile the file
!edgetpu_compiler -s /content/yolov5/yolov5s-int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 65 ms.

Input model: /content/yolov5/yolov5s-int8.tflite
Input size: 7.58MiB
Output model: yolov5s-int8_edgetpu.tflite
Output size: 7.51MiB
On-chip memory used for caching model parameters: 0.00B
On-chip memory remaining for caching model parameters: 8.09MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 197
Operation log: yolov5s-int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 1
Number of operations that will run on CPU: 196

Operator                       Count      Status

ADD                            15         More than one sub

In [ ]:
#Download models (fp16, int8)
from google.colab import files
import glob

%cd /content/yolov5/

for file in glob.glob('./*tpu.tflite'):
   files.download(file)

/content/yolov5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>